<a href="https://colab.research.google.com/github/riverdogcabin/PSDS4900/blob/main/wunderground_retrieve_observations_by_date.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import requests
import json
from pprint import pprint
import pandas as pd
from datetime import datetime
from datetime import timedelta
from time import sleep
import ipywidgets as widgets
# from ipywidgets import Display

In [22]:
start_date = widgets.DatePicker(description="Start Date")
end_date = widgets.DatePicker(description = "End Date")
display(start_date)
display(end_date)

DatePicker(value=None, description='Start Date')

DatePicker(value=None, description='End Date')

In [24]:
numdays  = end_date.value - start_date.value
date_list = [end_date.value - timedelta(days=x) for x in range(numdays.days+1)]


In [25]:
# Set up API call URL
# weatherreadings = {}
url = 'https://api.weather.com/v2/pws/history/hourly?stationId={stationid}&format=json&units={units}&date={date}&apiKey={apikey}&numericPrecision={numeric_precision}'
# url = 'https://api.weather.com/v2/pws/observations/hourly/7day?stationId={stationid}&format=json&units={units}&apiKey={apikey}&numericPrecision={numeric_precision}'
# url = "https://api.weather.com/v2/pws/observations/current?stationId=REPLACESTATIONID&format=json&units=REPLACEUNITS&numericPrecision=decimal&apiKey=REPLACEAPI"


In [26]:
# Get Config Data from LinearRegression.config
with open('/content/drive/MyDrive/Capstone/PSDS4900/LinearRegression.config') as configuration:
  config = json.load(configuration)
  model_location = config.get('model') #location of the model
  stations = config.get('stations')
  columns_to_pull = config.get('columns_to_pull')

# Get Config Data from config.json
with open('/content/drive/MyDrive/Capstone/PSDS4900/config.json') as configuration:
  config = json.load(configuration).get('WU') #config for WeatherUnderground
  apikey = config["apikey"]
  my_stationid = config["stationid"]
  units = config["units"]
  numeric_precision = config["numeric_precision"]  


In [27]:
url = url.format(stationid = '{stationid}',date='{date}',units=units,apikey=apikey,numeric_precision=numeric_precision)
drop_columns = ['lat','lon','units','tz','obsTimeUtc','obsTimeLocal']

In [28]:
# JSON comes back with measurements nested inside 'metrics', need to flatten
def flatten_wu_result(wu_result,units='metric'):
  try:
    observation = wu_result.pop(units)
    wu_result.update(observation)
    wu_result.update({'units':units})
  except:
    return wu_result

In [29]:
# function to get the JSON data from Weather Underground
def get_wu_data_specific_date(stationid,date,url):
  url = url.format(stationid=stationid,date=date)
  print("Retrieving data for {} and date {}".format(stationid,date),end=': ')
  response = requests.get(url) # need to add error handling for invalid data

  if response.status_code == 200: # If the response code is 200 (sucessful)
      print("Success")
      weatherdata = response.json()
      # pprint(weatherdata)
      for observation in weatherdata.get('observations'):
        flatten_wu_result(observation)
      return weatherdata.get('observations')
  elif response.status_code == 204: # 204 returns a blank response so inform the user
      print("API Error 204: No Data Found for specific query.")
      # exit()

  elif response.status_code >200 & response.status_code <1000: # Handles all other error code numbers
      # result = response.json()
      try:
          print("API Error " + str(response.status_code) + ": " + response.text)
      except: # sometimesthe error messae is in a different nest
          print("API Error " + str(response.status_code) + ": " + result["errors"][0]["message"])
      exit()

  else: # Catch any other errors
      print("Unspecified error")
      print(response.text)

In [ ]:
result_dict = {}
for date in date_list:
  result_dict[date.strftime("%Y%m%d")] = []
  for station in stations:
    result_dict[date.strftime("%Y%m%d")].append(station)
# result_dict

In [36]:
observations_df = pd.DataFrame()
for date in result_dict.keys():
  for station in result_dict.get(date):
    observations_df = observations_df.append(pd.DataFrame(get_wu_data_specific_date(station,date,url)))
    sleep(1)
  sleep(10)

Retrieving data for KCOCASTL161 and date 20210131: Success
Retrieving data for KCOCASTL208 and date 20210131: Success
Retrieving data for KCOCASTL308 and date 20210131: Success
Retrieving data for KCOCASTL321 and date 20210131: Success
Retrieving data for KCOCASTL397 and date 20210131: Success
Retrieving data for KCOCASTL399 and date 20210131: Success
Retrieving data for KCOPARKE148 and date 20210131: Success
Retrieving data for KCOPARKE26 and date 20210131: Success
Retrieving data for KCOPARKE271 and date 20210131: Success
Retrieving data for KCOPARKE276 and date 20210131: Success
Retrieving data for KCOPARKE305 and date 20210131: Success
Retrieving data for KCOPARKE313 and date 20210131: Success
Retrieving data for KCOPARKE320 and date 20210131: Success
Retrieving data for KCOPARKE331 and date 20210131: Success
Retrieving data for KCOSILVE32 and date 20210131: Success
Retrieving data for KCOCASTL317 and date 20210131: Success
Retrieving data for KCOCASTL79 and date 20210131: Success


In [37]:
if(observations_df.shape == (0,0)):
  print("Observations unavailable for these stations on these dates")
else:
  # observations_df.drop(columns=drop_columns,inplace=True)
  observations_df.to_csv('/content/drive/MyDrive/Capstone/PSDS4900/wu_data/predictions_observations_{}_{}.csv'.format(start_date.value.strftime('%Y%m%d'),end_date.value.strftime('%Y%m%d')),index=False)
  print('/content/drive/MyDrive/Capstone/PSDS4900/wu_data/predictions_observations_{}_{}.csv'.format(start_date.value.strftime('%Y%m%d'),end_date.value.strftime('%Y%m%d')))  

/content/drive/MyDrive/Capstone/PSDS4900/wu_data/predictions_observations_20210101_20210131.csv


In [38]:
observations_df.shape

(18758, 38)

In [39]:
observations_df.head()

,stationID,tz,obsTimeUtc,obsTimeLocal,epoch,lat,lon,solarRadiationHigh,uvHigh,winddirAvg,humidityHigh,humidityLow,humidityAvg,qcStatus,tempHigh,tempLow,tempAvg,windspeedHigh,windspeedLow,windspeedAvg,windgustHigh,windgustLow,windgustAvg,dewptHigh,dewptLow,dewptAvg,windchillHigh,windchillLow,windchillAvg,heatindexHigh,heatindexLow,heatindexAvg,pressureMax,pressureMin,pressureTrend,precipRate,precipTotal,units
0,KCOCASTL161,America/Denver,2021-01-31T07:59:55Z,2021-01-31 00:59:55,1612079995,39.431744,-104.787292,0,0,131.0,79.0,75.0,77.5,1,-5.8,-7.1,-6.5,0.0,0.0,0.0,0.0,0.0,0.0,-9.3,-10.3,-9.8,-5.8,-7.1,-6.5,-5.8,-7.1,-6.5,899.42,898.75,0.68,0.0,0.0,metric
1,KCOCASTL161,America/Denver,2021-01-31T08:59:55Z,2021-01-31 01:59:55,1612083595,39.431744,-104.787292,0,0,132.0,82.0,78.0,79.8,1,-6.2,-7.6,-7.0,2.5,0.0,0.1,4.0,0.0,0.2,-9.3,-10.5,-10.0,-6.2,-7.6,-7.0,-6.2,-7.6,-7.0,899.76,899.09,0.34,0.0,0.0,metric
2,KCOCASTL161,America/Denver,2021-01-31T09:59:07Z,2021-01-31 02:59:07,1612087147,39.431744,-104.787292,0,0,75.0,83.0,78.0,81.4,1,-6.4,-7.9,-7.3,1.4,0.0,0.0,4.0,0.0,0.1,-9.1,-10.6,-9.9,-6.4,-7.9,-7.3,-6.4,-7.9,-7.3,900.44,899.76,0.69,0.0,0.0,metric
3,KCOCASTL161,America/Denver,2021-01-31T10:59:40Z,2021-01-31 03:59:40,1612090780,39.431744,-104.787292,0,0,141.0,86.0,79.0,82.3,1,-6.5,-8.5,-7.4,0.0,0.0,0.0,0.0,0.0,0.0,-9.3,-10.7,-9.9,-6.5,-8.5,-7.4,-6.5,-8.5,-7.4,901.46,900.44,0.35,0.0,0.0,metric
4,KCOCASTL161,America/Denver,2021-01-31T11:59:56Z,2021-01-31 04:59:56,1612094396,39.431744,-104.787292,0,0,125.0,85.0,78.0,83.1,1,-6.5,-8.4,-7.9,7.2,0.0,0.7,7.9,0.0,0.9,-9.5,-11.0,-10.3,-6.5,-11.9,-8.1,-6.5,-8.4,-7.9,902.13,901.12,1.02,0.0,0.0,metric
